In [1]:
import urllib.request as url
import ssl
from bs4 import BeautifulSoup as BS

# Obter atas do Copom

## Código para transformar os pdfs em texto

Criado a partir da modificação de:

http://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

def convert_pdf(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    process_pdf(rsrcmgr, device, fp)
    fp.close()
    device.close()

    str = retstr.getvalue()
    retstr.close()
    return str

## Baixar as atas do COPOM

Para ignorar a verificação de certificados, me baseio em:
http://stackoverflow.com/questions/36600583/python-3-urllib-ignore-ssl-certificate-verification

In [4]:
#Ignorar a verificação de certificados no "urlopen"
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

Aqui, se a ata estiver em pdf, utilizo o programa acima para transformar em um txt. Se for um html, transformo em um txt diretamente.

In [20]:
atas = range(200,201)
path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/"

for ata in atas:
    ata = str(ata)
    link = "https://www.bcb.gov.br/?COPOM" + ata

    #Obter o arquivo
    filename = path + "COPOM_" + ata
    site = url.urlopen(link,context=ctx)
    content_type = dict(site.getheaders())["Content-Type"]
    
    #Checar se é um PDF ou um HTML
    if "pdf" in content_type:

        #Baixar o arquivo
        with open(filename + ".pdf","wb") as file_in:
            file_in.write(site.read())
        file_in.close()
        
        full_text = convert_pdf(filename + ".pdf")

        #Escrever no arquivo txt
        file_out = open(filename + ".txt","w")
        file_out.write(full_text[full_text.index("Data:"):])
        file_out.close()
        
    else:
        soup = BS(site.read(),"lxml")
        full_text = "\n".join(soup.findAll(text=True))
        
        #Escrever no arquivo txt
        file_out = open(filename + ".txt","w")
        file_out.write(full_text[full_text.index("Data:"):])
        file_out.close()